In [1]:
# !pip install setuptools

In [2]:
# !pip install -r requirements.txt

In [3]:
# Tengo un dataset en data/train/ que son imágenes para entrenar un modelo de clasificación de redes neuronales convolucionales
# Quiero que el modelo sea capaz de clasificar imágenes de gatos y perros

# 1. Cargar las imágenes
# 2. Preprocesar las imágenes
# 3. Crear el modelo
# 4. Entrenar el modelo
# 5. Evaluar el modelo
# 6. Guardar el modelo
# 7. Hacer predicciones

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers, models, regularizers


In [4]:
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

GPUs disponibles: 0


In [5]:
train_dir = "data/train"
test_dir = "data/test"

# Carga los datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    labels='inferred',         # Las etiquetas se infieren del nombre de la carpeta
    label_mode='int',          # Etiquetas como enteros
    image_size=(128, 128),     # Redimensiona las imágenes a 128x128
    batch_size=32,             # Tamaño del lote
    shuffle=True               # Mezcla los datos
)

test_dataset = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    shuffle=False
)
# Inspección de los datos
class_names = train_dataset.class_names
print(f"Clases detectadas: {class_names}")

Found 4733 files belonging to 2 classes.
Found 1184 files belonging to 2 classes.
Clases detectadas: ['chihuahua', 'muffin']


In [6]:

# Opcional: Normalización de imágenes (valores entre 0 y 1)
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))



# Opcional: Configuración para prefetching (mejora el rendimiento durante el entrenamiento)
# AUTOTUNE = tf.data.AUTOTUNE
# train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


In [7]:
# Definir el modelo de la CNN
model = models.Sequential([
    # Primera capa convolucional
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),

    # Segunda capa convolucional
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Tercera capa convolucional
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Aplanar y agregar capas densas
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),

    # Capa de salida (número de clases)
    layers.Dense(len(class_names), activation='softmax')  # len(class_names) = número de clases
])

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model.summary()

# Entrenar el modelo
model.fit(train_dataset, epochs=10)

# Evaluar el modelo
loss, accuracy = model.evaluate(test_dataset)
print(f"Pérdida: {loss}")
print(f"Exactitud: {accuracy}")


c:\Magister\3er trimestre\MLA\MLA_T2\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,313,026 (12.64 MB)

 Trainable params: 3,313,026 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 30s 195ms/step - accuracy: 0.6444 - loss: 0.6589
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 26s 176ms/step - accuracy: 0.8424 - loss: 0.3593
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 26s 173ms/step - accuracy: 0.8643 - loss: 0.3073
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 26s 175ms/step - accuracy: 0.9051 - loss: 0.2372
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 26s 175ms/step - accuracy: 0.9137 - loss: 0.2026
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 26s 176ms/step - accuracy: 0.9360 - loss: 0.1602
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 26s 175ms/step - accuracy: 0.9514 - loss: 0.1242
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 28s 191ms/step - accuracy: 0.9574 - loss: 0.1052
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 159ms/step - accuracy: 0.9700 - loss: 0.0793
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 23s 158ms/step - accuracy: 0.9801 - loss: 0.0514
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.8952 - loss: 0.3680
Pérdida: 0.34935054183006287
Exacti

In [8]:
model_comb1 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),  # Dropout en la primera capa convolucional

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),  # Dropout en las capas densas
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),

    layers.Dense(len(class_names), activation='softmax')
])

# Compilar el modelo
model_comb1.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model_comb1.summary()

# Entrenar el modelo
model_comb1.fit(train_dataset, epochs=10)

# Evaluar el modelo
loss, accuracy = model_comb1.evaluate(test_dataset)
print(f"Pérdida: {loss}")
print(f"Exactitud: {accuracy}")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,314,690 (12.64 MB)

 Trainable params: 3,313,858 (12.64 MB)

 Non-trainable params: 832 (3.25 KB)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 59s 373ms/step - accuracy: 0.7330 - loss: 0.6624
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 56s 374ms/step - accuracy: 0.8612 - loss: 0.3369
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 55s 372ms/step - accuracy: 0.8787 - loss: 0.3012
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 56s 376ms/step - accuracy: 0.8827 - loss: 0.2972
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 55s 373ms/step - accuracy: 0.9116 - loss: 0.2289
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 56s 375ms/step - accuracy: 0.9096 - loss: 0.2278
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 56s 374ms/step - accuracy: 0.9317 - loss: 0.1840
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 55s 373ms/step - accuracy: 0.9309 - loss: 0.1792
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 56s 374ms/step - accuracy: 0.9451 - loss: 0.1419
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 56s 375ms/step - accuracy: 0.9465 - loss: 0.1430
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.9272 - loss: 0.1748
Pérdida: 0.4451979696750641
Exactit

In [9]:
model_comb2 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),

    layers.Dense(len(class_names), activation='softmax')
])

# Optimizer con decay
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
model_comb2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model_comb2.summary()

# Entrenar el modelo
model_comb2.fit(train_dataset, epochs=10)

# Evaluar el modelo
loss, accuracy = model_comb2.evaluate(test_dataset)

print(f"Pérdida: {loss}")
print(f"Exactitud: {accuracy}")

c:\Magister\3er trimestre\MLA\MLA_T2\.venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,314,690 (12.64 MB)

 Trainable params: 3,313,858 (12.64 MB)

 Non-trainable params: 832 (3.25 KB)

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 2)

In [ ]:
model_comb3 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),  # Dropout en las capas convolucionales

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),  # Dropout en las capas densas
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),

    layers.Dense(len(class_names), activation='softmax')
])

# Optimizer con decay
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
model_comb3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model_comb3.summary()

# Entrenar el modelo
model_comb3.fit(train_dataset, epochs=10)

# Evaluar el modelo
loss, accuracy = model_comb3.evaluate(test_dataset)

print(f"Pérdida: {loss}")

print(f"Exactitud: {accuracy}")